# Some shit with randomization

I used numbers to get 10 families with <=5 sequences

In [109]:
import os

In [115]:
#os.chdir("./..")
#!pwd

/home/julybel/py_proj/families


##### Choose the numbers of families

In [116]:
gen_range, numbers, amount_of_fam = int(input('Print number RANGE')), [], int(input('Print size of the sample'))

while amount_of_fam > 0:
    number = random.randint(1, gen_range)
    if number not in numbers:
        numbers.append(number)
        amount_of_fam -= 1

Print number RANGE3115
Print size of the sample10


##### Downloading files

In [117]:
with open("files_to_download.txt", 'w') as out:
    for num in numbers:
        out.write('ftp://ftp.ebi.ac.uk/pub/databases/Rfam/14.1/fasta_files/RF' + ((5 - len(str(num))) * '0' + str(num)) + '.fa.gz' + '\n')
! wget -i "files_to_download.txt" -P "./newfiles/"

--2019-05-04 18:04:25--  ftp://ftp.ebi.ac.uk/pub/databases/Rfam/14.1/fasta_files/RF00176.fa.gz
           => «./newfiles/RF00176.fa.gz»
Распознаётся ftp.ebi.ac.uk (ftp.ebi.ac.uk)… 193.62.192.4
Подключение к ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.192.4|:21... соединение установлено.
Выполняется вход под именем anonymous … Выполнен вход в систему!
==> SYST ... готово.    ==> PWD ... готово.
==> TYPE I ... готово.  ==> CWD (1) /pub/databases/Rfam/14.1/fasta_files ... готово.
==> SIZE RF00176.fa.gz ... 1051
==> PASV ... готово.    ==> RETR RF00176.fa.gz ... готово.
Размер (байт): 1051 (1,0K) (не достоверно)

RF00176.fa.gz       100%[===================>]   1,03K  --.-KB/s    за 0,003s  

2019-05-04 18:04:27 (324 KB/s) - «./newfiles/RF00176.fa.gz» сохранён [1051]

--2019-05-04 18:04:27--  ftp://ftp.ebi.ac.uk/pub/databases/Rfam/14.1/fasta_files/RF00305.fa.gz
           => «./newfiles/RF00305.fa.gz»
Подключение к ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.192.4|:21... соединение установлено.
Выполняе

##### Unpacking and deleting archives

In [119]:
os.chdir("./newfiles/")
! gunzip *.gz

##### Here we determine the number of seq form each family (<=5); in case FOR 10 FAMILIES

In [122]:
! grep -c "^>" *.fa >> number_of_seqs.txt
with open("number_of_seqs.txt", 'r') as fname:
    num_of_seq = []
    for num in fname:
        num_of_seq.append(int(num.strip().split(":")[1]))
    need_num = min(num_of_seq) if min(num_of_seq) <= 5 else 5

In [123]:
need_num

5

##### Now we make our sample :) WITH UNIQUE-CHECKER

In [143]:
'''
If there're ununique values, we'll reduce our sample
So, please, check proportion of families in sample.fasta
'''
# need_num
# >RNU6-511P_ENSG00000200366.1,0
import glob
from random import sample
from Bio import SeqIO

out = open("sample.fasta", 'a')
fam_name = 0

for file in glob.glob("./*.fa"):
    fam = file[2:-3]
    #record = SeqIO.to_dict(SeqIO.parse(file, "fasta"))
    record, seq_check = {}, []
    for rec in SeqIO.parse(file, "fasta"):
        #print(rec.id, rec.seq)
        if rec.id not in record and str(rec.seq) not in seq_check:
            record[rec.id] = str(rec.seq)
            seq_check.append(str(rec.seq))
        #print(record)
    
    if need_num > len(record):
        end = len(record)
    else:
        end = need_num
        
    sp = random.sample(record.keys(), end)
    
    for seq_name in sp:
        out.write(">" + fam + "_" + str(seq_name).split("/")[0] + "," + str(fam_name) + '\n')
        out.write(str(record[seq_name]) + '\n')
    fam_name += 1
    
out.close()

In [157]:
! grep -c ",10" sample.fasta >> HOWMUCHFAM.txt

In [94]:
! grep -c "^>" sample.fasta

40


##### Making genelabel file

In [158]:
from Bio import SeqIO
with open("genelabel_sample.txt", 'a') as out:
    for record in SeqIO.parse("sample.fasta", "fasta"):
        iden = str(record.id).split(",")
        out.write(iden[0] + ":" + iden[1] + '\n')

In [159]:
! grep -c ":" genelabel_sample.txt

50
